<a href="https://colab.research.google.com/github/gcivitate/Python/blob/main/JK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import glob
import pandas as pd
import numpy
import scipy
import statsmodels
import sklearn
from datetime import date, time, datetime, timedelta
from dateutil.relativedelta import relativedelta
from pandas.core.groupby import NamedAgg

In [ ]:
# Ler os arquivos report_ que terminam com *xlsx
def load_report():
    path = '/content'
    test = pd.DataFrame()
    filenames = glob.glob(path + "/*.xlsx")


    for df in filenames:
        df_report = pd.read_excel(df)
        test = test.append(df_report,ignore_index = True)
    return test

def load_view():
  """
  Função que lê importa o arquivo exportado da view v_dados_cli_maq_jkcontrol
  """
  df = pd.read_csv("/content/v_dados_cli_maq_jkcontrol.csv", encoding="ISO-8859-1")
  return df

def load_maquina():
  """
  Função que lê o arquivo máquina operador
  """
  df = pd.read_csv("/content/maquina_01_device_rfid_variable_Gs5M.csv")
  return df

def treatment_maquina(df_maquina):
  """
  Função que faz o tratamento do dataframe do df_maquina.
  """
  # Trata o Json dentro da coluna context_rfid
  values = df_maquina.context_rfid.str.replace("'", '"').apply(json.loads).tolist()
  df = pd.DataFrame(values)

  df = pd.concat([df, df_maquina],axis = 1, join = 'outer', ignore_index=False, sort=False)

  df = df.drop('context_rfid', axis=1)

  # Separa o código do nome do funcionario
  df['cod_fun'] = df['cod_fun'].str.replace('-','')

  df_func = df['cod_fun'].str.split('(\d+)([A-Za-z]+)', expand=True)
  df_func = df_func.loc[:,[1,2]]
  df_func.rename(columns={1:'cod_func', 2:'name_func'}, inplace=True)

  df = df.drop('cod_fun', axis=1)

  df = pd.concat([df_func, df],axis = 1, join = 'outer', ignore_index=False, sort=False)

  # Renomear a coluna date
  df = df.rename(columns={'date (America/Sao_Paulo)': 'date'})

  return df

df_view = load_view()
df_report = load_report()
df = load_maquina()
df_maquina = treatment_maquina(df)


In [ ]:
#Verificando quantidade de paradas por tipo:

cod_parada = df_maquina.groupby('cod_maq') \
            .agg(n = pd.NamedAgg('cod_maq', 'count')) \
            .sort_values(by='n')

cod_parada

,n
cod_maq,
25,1
1,2
160,2
13,2
17,2
3,2
8,2
1,3
6,4


In [ ]:
#Verificando quantidade de parada por funcionario
count_func = df_maquina.groupby('cod_func') \
            .agg(n = pd.NamedAgg('cod_func', 'count')) \
            .sort_values(by='n')

count_func

,n
cod_func,
62,6
66,7
11,12
34,14
69,19
98,20
21,20
105,26
77,27


# ------------------- MÁQUINA 1 -----------------------

In [ ]:
#Tipos de produtos produzidos pela maq 1
count_prd = df_maquina.groupby('cod_prd') \
            .agg(n = pd.NamedAgg('cod_prd', 'count')) \
            .sort_values(by='n')

count_prd

,n
cod_prd,
12345678910111,1
idn,1
idn1111,1
IDN1111,54


In [ ]:
# Media ciclo maquina 1 em segundos
media_df_report = (df_report['ciclo (maquina_01 chen hsong)'].mean()) / 1000
media_df_report = round(media_df_report,2)
print(f"Media do ciclo total da maquina chen hsong: {media_df_report}")

Media do ciclo total da maquina chen hsong: 39.82


In [ ]:
# Desvio médio absoluto ciclo máquina 1

df_report \
          .agg(mad_ciclo = pd.NamedAgg('ciclo (maquina_01 chen hsong)', 'mad')) \
          .dropna(thresh=1) \
          .reset_index()

,index,ciclo (maquina_01 chen hsong)
0,mad_ciclo,3940.587515


In [ ]:
# Variância e desvio padrão do ciclo maquina 1

df_report \
          .agg(variancia_ciclo = pd.NamedAgg('ciclo (maquina_01 chen hsong)', 'var'), dp_ciclo = pd.NamedAgg('ciclo (maquina_01 chen hsong)', 'std')) \
          .reset_index()

,index,ciclo (maquina_01 chen hsong)
0,variancia_ciclo,1.376449e+10
1,dp_ciclo,1.173222e+05


In [ ]:
# Total de peças com defeito da maq 1

pc_defeito = df_report['peca_defeito (maquina_01 chen hsong)'].sum()
pc_defeito = round(pc_defeito)
print(f"Número de peças produziadas com defeitos: {pc_defeito}")

Número de peças produziadas com defeitos: 153


In [ ]:
# Qtde de ciclos com defeito da maq 1

pc_defeito = df_report['peca_defeito (maquina_01 chen hsong)'].count()
print(f"Quantidade de produção com defeito: {pc_defeito}")

Quantidade de produção com defeito: 14


In [ ]:
# Qtde de ciclos maq 1

qtd_ciclos = df_report['ciclo (maquina_01 chen hsong)'].count()
print(f"Qtdde ciclos: {qtd_ciclos}")

Qtdde ciclos: 19544


In [ ]:
# Total de peças produzidas pela maq 1

injecao = df_report['injecao (maquina_01 chen hsong)'].sum()
injecao = round(injecao)
print(f'Total de peças produzidas: {injecao}')

Total de peças produzidas: 78198


In [ ]:
# Pico de Produção maq 1

injecao = df_report['injecao (maquina_01 chen hsong)'].max()
injecao = round(injecao)
print(f'Pico máximo de produção: {injecao}')

Pico máximo de produção: 2


# ------------------- MÁQUINA 15 -----------------------

In [ ]:
# Qtde de ciclos maq 15

qtd_ciclos = df_report['ciclo (maquina_15 Haitian)'].count()
print(f"Qtdde ciclos: {qtd_ciclos}")

Qtdde ciclos: 37847


In [ ]:
# Media ciclo maquina 15 em segundos
media_df_report = (df_report['ciclo (maquina_15 Haitian)'].mean()) / 1000
media_df_report = round(media_df_report,2)
print(f"Media do ciclo total da maquina Haitian: {media_df_report}")

Media do ciclo total da maquina Haitian: 18.63


In [ ]:
# Desvio médio absoluto ciclo máquina 15

df_report \
          .agg(mad_ciclo = pd.NamedAgg('ciclo (maquina_15 Haitian)', 'mad')) \
          .dropna(thresh=1) \
          .reset_index()

,index,ciclo (maquina_15 Haitian)
0,mad_ciclo,7395.658103


In [ ]:
# Variância e desvio padrão do ciclo maquina 15

df_report \
          .agg(variancia_ciclo = pd.NamedAgg('ciclo (maquina_15 Haitian)', 'var'), dp_ciclo = pd.NamedAgg('ciclo (maquina_15 Haitian)', 'std')) \
          .reset_index()

,index,ciclo (maquina_15 Haitian)
0,variancia_ciclo,1.357642e+10
1,dp_ciclo,1.165179e+05


In [ ]:
# Peças com defeito - máquina 15

pc_defeito = df_report['peca_defeito (maquina_15 Haitian)'].sum()
pc_defeito = round(pc_defeito)
print(f"Número de peças produziadas com defeito: {pc_defeito}")

Número de peças produziadas com defeito: 695


In [ ]:
# Quantidade de ciclo com defeito máquina 15

pc_defeito = df_report['peca_defeito (maquina_15 Haitian)'].count()
print(f"Quantidade de ciclos com defeito: {pc_defeito}")

Quantidade de ciclos com defeito: 11


In [ ]:
# Total de peças produzidas máquina 15

injecao = df_report['injecao (maquina_15 Haitian)'].sum()
injecao = round(injecao)
print(f'Total de peças produzidas {injecao}')

Total de peças produzidas 1362619


In [ ]:
# Quantidade produzida pela máquina 15 - máximo de produção por Data\hora\segundo

injecao = df_report['injecao (maquina_15 Haitian)'].max()
injecao = round(injecao)
print(f'Pico máximo produzido informada pela máquina: {injecao}')

Pico máximo produzido informada pela máquina: 18


In [ ]:
pessoas=input("Quantas pessoas vão no casamento?")
valores=input("Qual o valor arrecadado?")
if valores<=800:
  print("Vamos para argentina")
elif valores<=400:
  print("Vamos para praia grande")
else:

